<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 11 08:41:55 2024

@author: Manasa Kesapragada
"""

import pandas as pd
import numpy as np
import math
from keras import backend as K
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.optimizers.legacy import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
import random
import tensorflow as tf
import seaborn as sns
import pdb
from keras.regularizers import l2

/Users/alexandranava/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Parameters

In [2]:
base = "/Users/alexandranava/Desktop/Spores/"
model_file = "/Users/alexandranava/Desktop/Spores/GerminationPrediction/initial_0_V4_Final.h5"

In [3]:
df_test = pd.read_csv("/Users/alexandranava/Desktop/Spores/M4581_s1/Analysis/V3/M4581_s1_Model_Data_V2.csv")
df_train = pd.read_csv("/Users/alexandranava/Desktop/Spores/M4576_s2/M4576_s2_Model_Data_V2.csv")

Initialize Data

In [4]:
columns_to_drop = ['X_POSITION', 'Y_POSITION']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)
#df['drug'] = df['drug'].fillna(0)

# Reset particle IDs to start from 1
df_train['new_spore_id'], unique_ids = pd.factorize(df_train['SPORE_ID'])
df_train['new_spore_id'] = df_train['new_spore_id'] + 1

df_test['new_spore_id'], unique_ids = pd.factorize(df_test['SPORE_ID'])
df_test['new_spore_id'] = df_test['new_spore_id'] + 1

# Drop index columns
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

#plt.hist(df_train['PERIMETER'])
input_cols = ['INTENSITY','AREA','GERMINANT_EXPOSURE','ELLIPSE_MINOR', "PERIMETER", 'GERMINATION']
output_cols = 'GERMINATION'


### Sensitivity Analysis
https://youtu.be/3wNxZcvRdPI?si=anEn6esqAH6it_nN

local sensitivity = $\frac{\Delta Y}{\Delta X_i}$ for parameters $X_i$

In [5]:
lookback = 3
def create_dataset(df, cells, lookback, in_cols= input_cols, out_cols=output_cols, drug_conc=[1]):
    trainX, trainY = [], [] #lists of training and testing inputs/outputs
    for drug in drug_conc:
        for track in range(cells[0], cells[1]):
            cell = df.loc[(df["new_spore_id"] == track)] #all rows of data pertaining to this cell
            cell = cell[in_cols] #reduce it to our columns of interest
            for i in range(len(cell)-lookback-1):
                trainX.append(cell[i:i+lookback])
            cell = cell[out_cols]
            #pdb.set_trace()
            for i in range(len(cell)-lookback-1):
                trainY.append(cell[i+lookback+1:i+lookback+2])

    trainX = np.array(list(map(lambda x: x.to_numpy(), trainX)))
    trainY = np.array(list(map(lambda x: x.to_numpy(), trainY)))
    return np.array(trainX), np.array(trainY)

# calculate data for train with 80% val with 20%
num_train_data = len(list(df_train["SPORE_ID"].unique()))
train_len = int(num_train_data * 0.85)
num_test_data = len(df_test["SPORE_ID"].unique())
print(f"training on spores (1, {train_len})...")
print(f"validating on spores ({train_len + 1}, {num_train_data})...")
print(f"testing on spores ({1, num_test_data})")

#training
trainX, trainY = create_dataset(df_train,cells=(1, train_len), lookback = 3)
valX, valY = create_dataset(df_train, cells=(train_len + 1, num_train_data), lookback = 3)
#testing
testX, testY = create_dataset(df_test, cells=(1, num_test_data), lookback = 3)


training on spores (1, 64)...
validating on spores (65, 76)...
testing on spores ((1, 54))


In [6]:
#trains numerous models using a list of numbers to initialize the models
def typical_model(trainX,trainY,valX,valY,testX,testY,numbers, model_file = None):
    models = [] #list of models
    predictions = [] #list of prediction vectors

    for i in numbers:
        
        # MODEL TRAINING
        if model_file == None: 
            print('Training model number {}'.format(i))
            model = Sequential()
            model.add(LSTM(80, input_shape=(trainX.shape[1], trainX.shape[2]))) #(Lookback of 1)
            model.add(Dense(1, activation='sigmoid'))
            model.add(Dropout(0.01))
            model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
            
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            
            history = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=30, batch_size=1, verbose=1, callbacks=[early_stopping])
            models.append(model)
            model.summary()

            model.save("initial_{}.h5".format(i))
            print("Saved model {} to disk".format(i))

            # PLOT TRAINING AND VALIDATION LOSS
            loss = history.history['loss']
            val_loss = history.history['val_loss']
            epochs = range(1, len(loss) + 1)
            
            plt.figure()
            plt.plot(epochs, loss, 'bo', label='Training loss')
            plt.plot(epochs, val_loss, 'b', label='Validation loss')
            plt.title('Training and validation loss for model {}'.format(i))
            plt.legend()
            plt.show()
            
            # PLOT TRAINING AND VALIDATION ACCURACY
            loss = history.history['accuracy']
            val_loss = history.history['val_accuracy']
            epochs = range(1, len(loss) + 1)
            
            plt.figure()
            plt.plot(epochs, loss, 'bo', label='Training accuracy')
            plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
            plt.title('Training and validation accuracy for model {}'.format(i))
            plt.legend()
            plt.show()        
        
        if model_file != None: 
            print("loading model...")
            model = load_model(model_file)
            models.append(model)


        #Predict on training, validation, and test sets
        trainPredict = model.predict(trainX)
        valPredict = model.predict(valX)
        testPredict = model.predict(testX)

        # Threshold the predictions to get binary outputs
        trainPredict_binary = (trainPredict > 0.5).astype(int)
        valPredict_binary = (valPredict > 0.5).astype(int)
        testPredict_binary = (testPredict > 0.5).astype(int)

        # Calculate RMSEs
        trainScore = math.sqrt(mean_squared_error(trainY, trainPredict_binary))
        print('Training RMSE: {}'.format(trainScore))
        valScore = math.sqrt(mean_squared_error(valY, valPredict_binary))
        print('Validation RMSE: {}'.format(valScore))
        testScore = math.sqrt(mean_squared_error(testY, testPredict_binary))
        print('Testing RMSE: {}'.format(testScore))

        # add predictions to list of prediction vectors
        preds = np.concatenate([trainPredict_binary, valPredict_binary, testPredict_binary], axis=None)
        predictions.append(preds)


    #find the average of all prediction vectors
    mean_pred = np.mean(predictions,axis=0)

    #find the prediction vector that is closest to the mean
    closest = 0 #index of the prediction vector that is closest to the mean
    dist = 100 #the distance of that closest vector to the mean vector
    for i in range(len(predictions)):
        thisdist = math.sqrt(mean_squared_error(predictions[i], mean_pred))

        print('Model number: {}, distance from mean: {}'.format(i,thisdist))

        if thisdist < dist:
            dist = thisdist
            closest = i

    #return the "most average" model
    print('Returning model {}, whose distance is {}'.format(closest,dist))
    return models, closest


models, closest = typical_model(trainX,trainY,valX,valY,testX,testY,range(1), model_file)
model = models[closest]
#model.save("alex_data_model_lk2.h5")
#model.save("alex_data_model_lk2.keras")
#print("Saved model to disk")


loading model...


  1/536 [..............................] - ETA: 2:06

 73/536 [===>..........................] - ETA: 0s  

149/536 [=======>......................] - ETA: 0s

232/536 [===========>..................] - ETA: 0s

...

275/473 [================>.............] - ETA: 0s

366/473 [======================>.......] - ETA: 0s

458/473 [============================>.] - ETA: 0s

473/473 [==============================] - 0s 551us/step


Training RMSE: 0.08574929257125442
Validation RMSE: 0.08574929257125442
Testing RMSE: 0.23791547571544325
Model number: 0, distance from mean: 0.0
Returning model 0, whose distance is 0.0


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
from scipy.stats import norm

plt.figure(12, 6)
for feature in input_cols:
  print(f"Feature: {feature}")
  for std_val in [0.1, 0.5, 1.0]:
    print(f"STD_factor = {std_val}")
    std_dev = (np.std(df_test[feature]) * std_val)
    print(f"{std_dev:.2f}")
    mean = 0 
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    y = norm.pdf(x, mean, std_dev)
    plt.plot(x, y, label=f'{std_val}$\sigma$')
  plt.legend()

TypeError: Value after * must be an iterable, not int

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

def local_sensitivity_analysis(df, feature: str, std_factor: int, use_feature_std: int):
    
    original_data = df.copy()
    original_color = "blue"
    perturbed_color = "orange"
    # Calculate mean and standard deviation
    feature_stddev: float = np.std(df[feature])
    feature_mean: float = np.mean(df[feature])

    x_min = np.min(df[feature]) - feature_stddev
    x_max = np.max(df[feature]) + feature_stddev
    
    print(rf"{feature.title()} has std = {feature_stddev}...")
    print(rf"{feature.title()} has mean = {feature_mean}...")
    
    # Choosing std to create Gaussian curve to produce noise
    if use_feature_std == 1:
        std = feature_stddev * std_factor
    else: 
        std = std_factor

    fig, ax = plt.subplots(1, 2, figsize=(14, 4))  # Changed to 2 subplots for distribution plots

    ### Adding Noise
    print(f"Using std = {std} to create Gaussian noise...")
    noise = np.random.normal(0, std, df[feature].shape)
    print(f"Adding {df[feature].shape} Gaussian noise samples from {np.min(noise)} to {np.max(noise)}...")
    df[feature] = df[feature] + noise
    
    #---------------PLOTS
    # Plot: Feature distribution before noise
    #ax[1].axvline(feature_mean, color='lightgrey', linestyle='--', label=r"$\mu$")
    #ax[1].axvline(feature_mean + feature_stddev, color='lightgrey', linestyle='--', label=rf"+{std_factor}$\sigma$")
    #ax[1].axvline(feature_mean - feature_stddev, color='lightgrey', linestyle='--', label=rf"-{std_factor}$\sigma$")
    
    ax[1].hist(original_data[feature], bins=30, alpha=0.25, color=original_color, label = "Original")
    ax[1].hist(df[feature], bins=30, alpha=0.25, color=perturbed_color, label = "Perturbed")
    ax[1].set_title(f"{feature.title()} Distribution")
    ax[1].set_xlabel('Value')
    ax[1].set_ylabel('Frequency')
    ax[1].set_xlim([x_min, x_max])
    ax[1].legend(loc="best")

    # Plot: Noise distribution plot
    ax[0].axvline(x=std, linestyle="--", color='lightgrey', label=rf"+$\sigma$")
    ax[0].axvline(x=-std, linestyle="--", color='lightgrey', label=rf"-$\sigma$")
    ax[0].hist(noise, bins=30, alpha=0.75, color='skyblue', edgecolor="black")
    ax[0].set_title(rf"Noise Distribution with $\sigma$={std:.2f}")
    ax[0].set_xlabel('Noise Value')
    ax[0].set_ylabel('Frequency')
    plt.legend()
    plt.show()

    #Plot: Feature before and after perturbation
    example_spore_perturbed = df[df["new_spore_id"] == 1]
    example_spore_original = original_data[original_data["new_spore_id"]==1]
    germination_status = example_spore_original["GERMINATION"].to_list()
    example_germination = germination_status.index(1)    
    plt.figure(figsize=(6, 4))

    sns.lineplot(x=range(0, len(germination_status)), y=example_spore_original[feature], color=original_color, label="Original")
    sns.lineplot(x=range(0, len(germination_status)), y=example_spore_perturbed[feature], color=perturbed_color, label="Perturbed", alpha = 0.5)
    
    plt.axvline(example_germination, color='lightgrey', linestyle='--', label="Germination")
    plt.xlabel("Frame")
    plt.ylabel(feature.title())
    plt.title(f"{feature.title()} over Time (Spore 1)")
    plt.legend(loc="best")
    plt.show()

    delta_x = np.mean(np.abs(original_data[feature] - df[feature]))

    return df, delta_x

sensitivity_feature = "PERIMETER"
feature_std: int = 0  # 0 to use std_factor directly, 1 to use feature_std * std_factor
scale_factor = 0.5  # Scale for std
df_test, delta_x = local_sensitivity_analysis(df_test, sensitivity_feature, scale_factor, feature_std)

In [ ]:
from SALib.sample.morris import sample
from SALib.analyze import morris
parameter_dict = {'num_vars': 6,
                  "names": ['INTENSITY','AREA','GERMINANT_EXPOSURE','ELLIPSE_MINOR', "PERIMETER", "GERMINATION"],
                  "bounds": [[np.min(df_test["INTENSITY"]), np.max(df_test["INTENSITY"])],
                              [np.min(df_test["AREA"]), np.max(df_test["AREA"])],
                              [np.min(df_test["GERMINANT_EXPOSURE"]), np.max(df_test["GERMINANT_EXPOSURE"])],
                              [np.min(df_test["ELLIPSE_MINOR"]), np.max(df_test["ELLIPSE_MINOR"])],
                              [np.min(df_test["PERIMETER"]), np.max(df_test["PERIMETER"])],
                              [np.min(df_test["GERMINATION"]), np.max(df_test["GERMINATION"])]]
                              }
perturbed_input = sample(parameter_dict, 2000)

#9am

In [ ]:
#Predict on training, validation, and test sets
trainPredict = model.predict(trainX)
trainPredict =(trainPredict > 0.5).astype(int)

valPredict = model.predict(valX)
valPredict =(valPredict > 0.5).astype(int)

testPredict = model.predict(testX)
testPredict =(testPredict > 0.5).astype(int)

#testPredict = model.predict(perturbed_input)
#testPredict = (testPredict > 0.5).astype(int)

$ \text{True Positive Rate} = \frac{\text{TP}}{\text{TP} + \text{FN}} $

$ \text{False Positive Rate} = \frac{\text{FP}}{\text{FP} + \text{TN}} $



$ \text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}} $

$ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} $

$ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} $

$ \text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} $


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy, precision, recall, and F1-score
train_accuracy = accuracy_score(trainY, trainPredict)
val_accuracy = accuracy_score(valY, valPredict)
test_accuracy = accuracy_score(testY, testPredict)
print("--------------------")
print("Accuracy:")
print('Training Accuracy: {}'.format(train_accuracy))
print('Validation Accuracy: {}'.format(val_accuracy))
print('Testing Accuracy: {}'.format(test_accuracy))

train_precision = precision_score(trainY, trainPredict)
val_precision = precision_score(valY, valPredict)
test_precision = precision_score(testY, testPredict)
print("--------------------")
print("Precision:")
print('Training Precision: {}'.format(train_precision))
print('Validation Precision: {}'.format(val_precision))
print('Testing Precision: {}'.format(test_precision))

train_recall = recall_score(trainY, trainPredict)
val_recall = recall_score(valY, valPredict)
test_recall = recall_score(testY, testPredict)
print("--------------------")
print("Recall:")
print('Training Recall: {}'.format(train_recall))
print('Validation Recall: {}'.format(val_recall))
print('Testing Recall: {}'.format(test_recall))

train_f1 = f1_score(trainY, trainPredict)
val_f1 = f1_score(valY, valPredict)
test_f1 = f1_score(testY, testPredict)
print("--------------------")
print("F1-Score:")
print('Training F1-Score: {}'.format(train_f1))
print('Validation F1-Score: {}'.format(val_f1))
print('Testing F1-Score: {}'.format(test_f1))

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve

# Plot ROC curve
def plot_roc_curve(y_true, y_pred, title):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    auc_score = roc_auc_score(y_true, y_pred)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (area = {auc_score:0.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
# Training set
train_auc = roc_auc_score(trainY, trainPredict)
print('Training AUC: {}'.format(train_auc))
plot_roc_curve(trainY, trainPredict, 'ROC curve - Training set')

In [ ]:
# Validation set
val_auc = roc_auc_score(valY, valPredict)
print('Validation AUC: {}'.format(val_auc))
plot_roc_curve(valY, valPredict, 'ROC curve - Validation set')

In [ ]:
# Test set
test_auc = roc_auc_score(testY, testPredict)
print('Testing AUC: {}'.format(test_auc))
plot_roc_curve(testY, testPredict, 'ROC curve - Testing set')

               Predicted
             |   0   |   1   |
    --------------------------
    True  0  |  TN   |  FP   |
    True  1  |  FN   |  TP   |


In [ ]:
# Confusion matrices
train_cm = confusion_matrix(trainY, trainPredict)
val_cm = confusion_matrix(valY, valPredict)
test_cm = confusion_matrix(testY, testPredict)
print('Training Confusion Matrix:')
print(train_cm)
print('Validation Confusion Matrix:')
print(val_cm)
print('Testing Confusion Matrix:')
print(test_cm)

In [ ]:
print('Writing results for model {}'.format(1))
maxtrack = int(max(df_test['new_spore_id']))
for track in range(1, maxtrack+1):
    print(f"track {track} of {maxtrack}...")
    cell = df_test.loc[(df_test['new_spore_id']==track)]
    if len(cell)==0:
        continue
    maxslice = max(df_test.loc[(df_test['new_spore_id']==track), 'FRAME'])+1
    minslice = min(df_test.loc[(df_test['new_spore_id']==track), 'FRAME'])
    for sl in range(int(minslice),int(maxslice+1)):
        x = cell.loc[(cell['FRAME']>sl-1) & (cell['FRAME']<=sl)]
        x = x[input_cols].to_numpy()
        if x.size > 0:
          x=x.reshape(1, 1, len(input_cols))
          df_test.loc[(df_test['new_spore_id']==track) & (df_test['FRAME']==sl), 'pred_germ{}'.format(1)] = model.predict(x)
 
df_test['pred_error{}'.format(1)] = df_test['pred_germ{}'.format(1)] - df_test['GERMINATION']
df_test['pred_germ1'] =(df_test['pred_germ1'] > 0.5).astype(int)


Plot of Predicted Spores Germinated against Actual Values

In [ ]:
plt.errorbar(df_test['FRAME'].unique(),df_test.groupby(['FRAME']).mean()['pred_germ1'],
            color = 'b', label = 'Predicted values')
plt.errorbar(df_test['FRAME'].unique(),df_test.groupby(['FRAME']).mean()['GERMINATION'],
             color = 'orange', label = 'Original values')
plt.yticks(fontsize=20)
plt.xlabel('Timestep',fontsize=14)
plt.ylabel('Mean spores germinated',fontsize=14)
plt.legend()

In [ ]:
# Get unique spore IDs
unique_spore_ids = df_test['new_spore_id'].unique()

# list of correct spores
spore_error: list[0, 1] = []

# Loop through each spore ID and create separate plots
for spore_id in unique_spore_ids:
    spore_data = df_test[df_test['new_spore_id'] == spore_id]
    
    # Create a new figure for each spore ID
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Plot GERMINATION values
    ax.errorbar(
        spore_data['FRAME'].unique(),
        spore_data.groupby(['FRAME']).mean()['GERMINATION'],
        yerr=spore_data.groupby(['FRAME']).sem()['GERMINATION'],
        color='red',
        linewidth = 5,
        label='Original GERMINATION values'
    )
    
    # Plot pred_germ1 values
    ax.errorbar(
        spore_data['FRAME'].unique(),
        spore_data.groupby(['FRAME']).mean()['pred_germ1'],
        yerr=spore_data.groupby(['FRAME']).sem()['pred_germ1'],
        color='black',
        linewidth = 3,
        label='Predicted GERMINATION values'
    )

    # Set plot title and labels
    ax.set_title(f'Spore ID {spore_id} - GERMINATION')
    ax.set_xlabel('FRAME')
    ax.set_ylabel('Values')
    ax.legend()

    #add 1 for correct and 0 for wrong
    binary_error = int((spore_data['GERMINATION'].mean() == spore_data['pred_germ1'].mean()))
    if binary_error == 0:
        print(spore_id)
    spore_error.append(binary_error)
    

    # Adjust layout
    plt.tight_layout()
    plt.show()


In [ ]:
#Accuracy on test data 
correct_percentage = (sum(spore_error) / len(spore_error)) * 100
print(f"{correct_percentage}% of spores predicted correctly...")
number_wrong = spore_error.count(0)
print(f"{number_wrong} spores incorrectly predicted...")

In [ ]:
if feature_std == 1:
    sensitivity_csv = f"{base}/Analysis/sensitivity_featurestd.csv"
if feature_std == 0:  # Use else instead of another if
    sensitivity_csv = f"{base}/Analysis/sensitivity_gaussian.csv"

accuracy = {
    "Feature": [sensitivity_feature],
    "Accuracy": [correct_percentage],
    "STD": [feature_std * scale_factor],
    "Delta x": [delta_x]
    }

accuracy_df = pd.DataFrame(accuracy)

# Ensure that a newline is correctly added before appending
# with open(sensitivity_csv, 'a') as f:
#     if os.path.getsize(sensitivity_csv) > 0:  # Check if file is non-empty
#         f.write('\n')  # Ensure a newline is added before appending

accuracy_df.to_csv(sensitivity_csv, index=False, mode="a", header=False)